# Get New Addresses

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import logging

import sys

sys.path.append("/app")

# import scraping as sc

import pandas as pd

from jinja2 import Template

from db_utils import get_engine, get_table_creation_query
import os

<IPython.core.display.Javascript object>

In [3]:
# global logger
logger = logging.getLogger()
handler = logging.StreamHandler()
formatter = logging.Formatter("%(asctime)s [%(name)s] %(levelname)-8s %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)

<IPython.core.display.Javascript object>

## Connect to Database

In [4]:
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
host = os.getenv("POSTGRES_HOST")

<IPython.core.display.Javascript object>

In [5]:
engine = get_engine(user, password, host)

<IPython.core.display.Javascript object>

## Define search parameters

In [6]:
searchname = "tamzin"

<IPython.core.display.Javascript object>

## Create unique addresses table

In [7]:
cols = {"address": "VARCHAR(256)", "property_id": "INTEGER", "url": "TEXT"}

unique_cols = ["address"]

index_cols = ["address", "property_id"]

<IPython.core.display.Javascript object>

In [8]:
create_q = get_table_creation_query(
    "unique_addresses", cols, searchname, index_cols, unique_cols
)

<IPython.core.display.Javascript object>

In [9]:
with engine.connect() as conn:
    conn.execute(create_q)

<IPython.core.display.Javascript object>

## Merge into Unique Addresses Table

In [28]:
q_merge_in_new = f"""INSERT INTO {searchname}.unique_addresses (address,property_id,url)
select distinct on (address) address,id,url
FROM {searchname}.new_data n
ORDER BY address,id desc
ON CONFLICT (address)
DO UPDATE
SET
    property_id=EXCLUDED.property_id,
    url=EXCLUDED.url
"""

<IPython.core.display.Javascript object>

In [29]:
with engine.connect() as conn:
    conn.execute(q_merge_in_new)

<IPython.core.display.Javascript object>

## Create completed table and invalid table

### Completed table

In [30]:
cols = {"address_id": "INTEGER"}

unique_cols = ["address_id"]

index_cols = ["address_id"]

<IPython.core.display.Javascript object>

In [31]:
create_q = get_table_creation_query(
    "completed_addresses", cols, searchname, index_cols, unique_cols
)

<IPython.core.display.Javascript object>

In [32]:
with engine.connect() as conn:
    conn.execute(create_q)

<IPython.core.display.Javascript object>

### Invalid table

In [33]:
create_q = get_table_creation_query(
    "invalid_addresses", cols, searchname, index_cols, unique_cols
)

<IPython.core.display.Javascript object>

In [34]:
with engine.connect() as conn:
    conn.execute(create_q)

<IPython.core.display.Javascript object>

## Create view for unprocessed addresses

In [35]:
q_view = f"""CREATE OR REPLACE VIEW {searchname}.address_ids_to_process AS
SELECT u.id as address_id
FROM {searchname}.unique_addresses u
WHERE
    (
    NOT EXISTS (
    SELECT  -- SELECT list mostly irrelevant; can just be empty in Postgres
    FROM   {searchname}.completed_addresses c
    WHERE  c.address_id = u.id
    )
    )
    AND
    (
    NOT EXISTS (
    SELECT  -- SELECT list mostly irrelevant; can just be empty in Postgres
    FROM   {searchname}.invalid_addresses i
    WHERE  i.address_id = u.id
    )
    )
"""

<IPython.core.display.Javascript object>

In [36]:
with engine.connect() as conn:
    conn.execute(q_view)

<IPython.core.display.Javascript object>